<h1><center> Analiza modelu naiwnego </center><h1>

Model działający naiwnie to model wykonujący predykcje jedynie na podstawie wartości czasu trwania danej sesji (w sekundach).
Jego wejście to aktualny czas trwania danej sesji w sekundach, a wyjście to decyzja czy zakończona zostanie ta sesja zakupem.

In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('seaborn')

In [35]:
df_sessions = pd.read_json(path_or_buf='../../data/raw/sessions.jsonl', lines=True)

In [36]:
df_sessions.head()

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id
0,124,2021-07-28 15:47:21,102,1283,VIEW_PRODUCT,5,NaN
1,124,2021-07-28 15:47:40,102,1283,BUY_PRODUCT,5,20001.0
2,125,2021-09-12 16:43:16,102,1076,VIEW_PRODUCT,0,NaN
3,125,2021-09-12 16:46:24,102,1080,VIEW_PRODUCT,0,NaN
4,125,2021-09-12 16:48:08,102,1079,VIEW_PRODUCT,0,NaN


In [37]:
df_sessions = df_sessions.drop(columns='purchase_id')

In [38]:
agg_func_sess_prod = {
    'timestamp' : lambda t: (t.max() - t.min()).seconds,
    'event_type' : lambda e: True if len(e.unique()) > 1 else False
}
agg_sess_prod_df = df_sessions.groupby(df_sessions['session_id']).aggregate(agg_func_sess_prod)
agg_sess_prod_df.rename(columns={'event_type' : 'purchased'}, inplace=True)
agg_sess_prod_df.rename(columns={'timestamp' : 'session_length'}, inplace=True)

In [39]:
agg_sess_prod_df

,session_length,purchased
session_id,,
124,19,True
125,1436,True
126,117,True
127,519,True
128,847,True
...,...,...
9646,0,False
9647,116,True
9648,1617,False


In [45]:
agg_sess_prod_df.purchased.sum()

5525

In [40]:
agg_sess_prod_df['session_length'].min()

0

In [43]:
agg_sess_prod_df.shape[0]

9527

In [44]:
max_accuracy = -1
best_threshold = -1
for n_seconds in range(agg_sess_prod_df['session_length'].min(), agg_sess_prod_df['session_length'].max()):
    naive_predictions = agg_sess_prod_df['session_length'] >= n_seconds
    current_accuracy = (naive_predictions==agg_sess_prod_df['purchased'].to_numpy()).sum()/agg_sess_prod_df.shape[0]
    if current_accuracy >= max_accuracy:
        best_threshold = n_seconds
        max_accuracy = current_accuracy
    print(f"For threshold = {n_seconds} accuracy = {current_accuracy:.3f}")

print(f"Best threshold is {best_threshold} with accuracy: {max_accuracy:.3f}")

For threshold = 0 accuracy = 0.580
For threshold = 1 accuracy = 0.696
For threshold = 2 accuracy = 0.696
For threshold = 3 accuracy = 0.696
For threshold = 4 accuracy = 0.696
For threshold = 5 accuracy = 0.696
For threshold = 6 accuracy = 0.695
For threshold = 7 accuracy = 0.694
For threshold = 8 accuracy = 0.694
For threshold = 9 accuracy = 0.693
For threshold = 10 accuracy = 0.693
For threshold = 11 accuracy = 0.692
For threshold = 12 accuracy = 0.692
For threshold = 13 accuracy = 0.691
For threshold = 14 accuracy = 0.690
For threshold = 15 accuracy = 0.690
For threshold = 16 accuracy = 0.689
For threshold = 17 accuracy = 0.688
For threshold = 18 accuracy = 0.688
For threshold = 19 accuracy = 0.687
For threshold = 20 accuracy = 0.686
For threshold = 21 accuracy = 0.684
For threshold = 22 accuracy = 0.683
For threshold = 23 accuracy = 0.682
For threshold = 24 accuracy = 0.681
For threshold = 25 accuracy = 0.681
For threshold = 26 accuracy = 0.680
For threshold = 27 accuracy = 0.680
Fo

Uznaliśmy, że model ten będzie działał najlepiej, gdy będzie klasyfikował jako kończące się zakupem wszystkie sesje, których czas trwania wynosi ponad 5 sekund.